# Advanced OOP

### Excursus/Review: Usage of \*args and \**kwargs
\*args and \**kwargs are mostly used in function definitions. These special constructs allow you to pass a variable number of arguments to a function. This means, that if you do not know beforehand how many arguments can be passed to a function by the user, then in this case you use these keywords.
Specifically, \*args is used to send a non-keyworded argument list to the function using a tuple, while \**kwargs sends the keyworded arguments collected into a dictionary.

In [ ]:
def add(a, b, c, d):
    print("Sum of a, b, c, d: ", a + b + c + d)
    
add(1, 2, 3, 4)

In [ ]:
add(1, 2, 3, 4, 5)

In [ ]:
def add2(*nums):
    print(nums)
    sum = 0
    for num in nums:
        sum = sum + num
    
    print("Sum of nums: ", sum)
    
add2(1, 2, 3, 4, 5)

In [ ]:
def add3(*nums):
    print(nums)
    print("Sum of sums: ", sum(nums))
    return sum(nums)
    
add3(1, 2, 3, 4, 5)

What if we want to reuse our "old" add functions?

In [ ]:
def add4(*nums):
    print(nums)
    print("Sum of sums: ", add3(*nums))
    
add4(1, 2, 3, 4, 5)

(*(1, 2, 3, 4, 5),)

In the example above - add3(*nums) "splices" in the nums-tuple, so that the individual arguments are provided to the function add3.

We can also "unpack" a dictionary, as captured by \**kwargs.

In [ ]:
d = {"bar" : "foo", "baz" : "hugo"}
dict(**d)

## \**kwargs

**kwargs works similarly to *args: It collects the keyword (variable length) into a dictionary, and provides that to our function ...

In [ ]:
def data(**the_data):
    print("Type of data: ", type(the_data))
    
    for key, value in the_data.items():
        print("{} is {}".format(key, value))
        
data(bla="blub", foo="bar", hugo="baz")

In [ ]:
def helper(**the_data):      
    print("Invoking our (intermediate) helper function ...")
    print(the_data)
    data(**the_data)
    
helper(bla="blub", foo="bar", hugo="baz")

In [ ]:
def data(**data):
    if data is not None:
        for key, value in data.items():
            print("{} is {}".format(key, value))
        
data(bla="blub", foo="bar", hugo="baz")

In [ ]:
class foo(object):
    def bar(self, a, b, c = False):
        if not c:
            print("C is False")
        else:
            print("C is True")
        pass

class foo2(object):
    def bar(self, a, b):
        super().bar(a, b)
        

class baz(foo, foo2):
    def bar(self, a, b, *args):
        super().bar(a, b, *args)
        
foo().bar(1, 2, True)
baz().bar(1, 2, True)

### Now: A more Complex decorator
As we have seen last week, decorators are a "built-in" design pattern in Python, using the "@decorator" syntax.

Below, we will target the following: A decorator to memoize functions with an arbitrary set of arguments (that is, we cache/memoize the results of the function for specific arguments). Note that memoization is only possible if the arguments are hashable (e.g. using their \__hash__() function). If our decorator wrapper is called with arguments which are not hashable, then the wrapped function should just be called without caching.

In [ ]:
def memoize(fun):
    fun.cache = {}
    def wrapper(*args, **kwargs):
        key = (args, tuple(sorted(kwargs.items())))
        try:
            result = fun.cache[key]
        except TypeError:
            # key is unhashable
            return fun(*args, **kwargs)
        except KeyError:
            # value is not (yet) present in our cache
            result = fun.cache[key] = fun(*args, **kwargs)
        return result
    
    return wrapper

@memoize
def oneplus(num):
    print("Calculcating 1+")
    return 1 + num

# oneplus = memoize(oneplus)

#wrapped_oneplus=memoize(oneplus)

oneplus(1)
oneplus(1)




* The function oneplus is provided to the memoize function
* We create the dictionary in the oneplus function object (fun.cache dictionary)
* We create the wrapper function, but it also has access to the oneplus function object
* The memoize function returns the wrapper (and the wrapper still knows about the original oneplus function)
 * For the name oneplus, we replace the reference to the function object by the reference to the wrapper function object. This means, that if we call oneplus(), we are actually calling the wrapper ...

## Functions as objects
Functions are objects, i.e., instances of class "function".

Therefore, also attributes can be added on the fly (see above)

In [ ]:
print(type(oneplus))
print(type(type(oneplus)))

oneplus.cache
oneplus.__dict__

## Creating instances

In [ ]:
import random

class KindOfBadSingleton(object):
    _inst = None
    def __new__(cls):
        print("__new__ {}".format(cls))
        if cls._inst is None:
            cls._inst = super().__new__(cls)
        return cls._inst
    def __init__(self):
        self.a = random.randrange(100)
        print("__init__ {}:{}".format(self, self.a))

KindOfBadSingleton()
KindOfBadSingleton()
KindOfBadSingleton()

KindOfBadSingleton


## Constructing Classes

We will look at this process - and see what is going on using a Metaclass.
Note: Metaclass here inherit from type (!).

(In principle, a metaclass could also be implemented using a function, but we will see this later ...)

Class construction
* \__prepare__()
* Execute class body
* Type instantiation, i.e., instance construction


Instance construction:
* cls.\__call__()
* obj.\__new__()
* obj.\__init__()

* Be careful mixing \__new__() and \__init__() - see above
* For the singleton, maybe cls.\__call__() may be more appropriate




In [ ]:
class PrintingMetaClass(type):
    @classmethod
    def __prepare__(mcls, name, bases, **kw):
        print("__prepare__", mcls, name, bases, kw)
        return super().__prepare__(name, bases, **kw)
    
    def __new__(mcls, name, bases, dct, **kw):
        print("__new__", mcls, name, bases, dct, kw)
        return super().__new__(mcls, name, bases, dct, **kw)
    
    def __init__(cls, name, bases, dct, **kw):
        print("__init__", cls, name, bases, dct, kw)
        return super().__init__(name, bases, dct, **kw)
    
class Spam(metaclass=PrintingMetaClass):
    print("Spam body")
    class Eggs(metaclass=PrintingMetaClass): pass

In [ ]:
class MyIntArray(type):
    def __getitem__(cls, index):
        if isinstance(index, slice):
            return list(map(MyInt, range(index.start, index.stop, index.step or 1)))
        else:
            return MyInt(index)

class MyInt(int, metaclass=MyIntArray):
    pass

print(MyInt[3:5])

print(MyInt(42))

### Better Singleton - using \__call__

In [ ]:
class BetterSingleton(type):
    _inst = None
    def __call__(cls, *p, **kw):
        if cls._inst is None:
            cls._inst = super().__call__(*p, **kw)
        return cls._inst
    
class BetterSingletonClass(metaclass=BetterSingleton):
    def __init__(self):
        print("__init__", self)
        
b1 = BetterSingletonClass()
b2 = BetterSingletonClass()

# Outlook: Functional Programming
* Functional programming - more abstract approach
* Program seen as evaluations of mathematical functions
* Functions as first-class objects
* Support for higher-order functions
* Recursion instead of loop constructs
* Lists as basic data structures
* Avoiding side effects (no shared state - immutable objects)


In [ ]:
print(list(filter(lambda x: x % 2 == 0, range(1,10))))

def test(x):
    return x % 2 == 0

print(list(filter(test, range(1,10))))

### Lambda
Defines an anonymous function
* No multiline lambdas
* Can be used instead of a function (see above)

In [ ]:
(lambda x: print(x))(1)
